In [77]:
# from IPython.utils import io
# with io.capture_output() as captured:
#     !pip install scispacy
#     !pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.4/en_core_sci_lg-0.2.4.tar.gz
#     !pip install whoosh

In [78]:
import whoosh
from whoosh.qparser import *
from whoosh.fields import Schema, TEXT, KEYWORD, ID, STORED
from whoosh.analysis import StandardAnalyzer
from whoosh import index

from whoosh.analysis import Tokenizer, Token
from whoosh import highlight

In [79]:
import pandas as pd
# df_Yahoo = pd.read_csv('Yahoo_Tickers.csv', encoding = "utf-8", na_values="\\N") 

In [80]:
# df_Yahoo.head()

In [81]:
# df_NASDAQ = pd.read_csv('NASDAQ_Tickers.csv')

In [82]:
# df_NASDAQ.head()

In [83]:
df = pd.read_csv('Tickers.csv', encoding = "utf-8", na_values="\\N") 
# df = pd.read_csv('SEC_Tickers.csv', encoding = "utf-8", na_values="\\N") 

In [84]:
df['Name'] = df['Name'].str.upper()

In [85]:
df_kek = pd.read_csv('kek.csv', encoding = "utf-8", na_values="\\N")

In [86]:
abbreviations = {
    'AG': 'AKTIENGESELLSCHAFT',
    'AVRG': 'AVERAGE',
    'BK': 'BANK',
    'BKG': 'BANKING',
    'CAP': 'CAPITAL',
    'CHEMS': 'CHEMICALS',
    'CO': 'COMPANY',
    'CONSTR': 'CONSTRUCTION',
    'CORP': 'CORPORATION',
    'CTLS': 'CONTROLS',
    'FINL': 'FINANCIAL',
    'FLA': 'FLORIDA',
    'FLTNG': 'FLOATING',
    'HLDG': 'HOLDING',
    'HLDGS': 'HOLDINGS',
    'INDL': 'INDUSTRIAL',
    'INTL': 'INTERNATIONAL',
    'LABS': 'LABORATORIES',
    'MACHS': 'MACHINES',
    'MATLS': 'MATERIALS',
    'MGMT': 'MANAGEMENT',
    'NATL': 'NATIONAL',
    'PAC': 'PACIFIC',
    'PETE': 'PETROLEUM',
    'PPTYS': 'PROPERTIES',
    'PRODS': 'PRODUCTS',
    'RLTY': 'REALTY',
    'RTE': 'RATE',
    'RY': 'RAILWAY',
    'SR': 'SENIOR',
    'SVCS': 'SERVICES',
    'TR': 'TRUST'
}

In [87]:
df_kek.head()

,Unnamed: 0,CIK,Name of fund,Quater of report,Signing date,Name of Issuer,Ticker of Issuer,CUSIP,Value (x$1000),Shares,Shares type,Investment Discretion,Voting Sole,Shared,None
0,0,1584087,1492 Capital Management LLC,2022-03-31,2022-05-11,A10 NETWORKS INC,ATEN,002121101,188585,10041,SH,SOLE,10041,0,0
1,1,1584087,1492 Capital Management LLC,2022-03-31,2022-05-11,ABBVIE INC,ABBV,00287Y109,140,1340,SH,SOLE,1340,0,0
2,2,1584087,1492 Capital Management LLC,2022-03-31,2022-05-11,ACCEL ENTERTAINMENT INC,acel,00436Q106,217,287328,SH,SOLE,287328,0,0
3,3,1584087,1492 Capital Management LLC,2022-03-31,2022-05-11,ADTRAN INC,ADTN,00738A106,3500,187092,SH,SOLE,187092,0,0
4,4,1584087,1492 Capital Management LLC,2022-03-31,2022-05-11,AMERESCO INC,AMRC,02361E108,3452,10401,SH,SOLE,10401,0,0


In [88]:
def replace_abbreviation(name):
    name = [abbreviations.get(word) if abbreviations.get(word) is not None else word for word in name.split()]
    name = ' '.join(name)
    return name

In [89]:
for index in range(len(df_kek)):
    df_kek['Name of Issuer'].iloc[index] = replace_abbreviation(df_kek['Name of Issuer'].iloc[index])

C:\Users\vladm\AppData\Local\Temp\ipykernel_44988\4170528891.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_kek['Name of Issuer'].iloc[index] = replace_abbreviation(df_kek['Name of Issuer'].iloc[index])


In [90]:
for index in range(len(df_kek)):
    df['Name'].iloc[index] = replace_abbreviation(df['Name'].iloc[index])


In [91]:
df

,Ticker,Name
0,OEDV,"OSAGE EXPLORATION AND DEVELOPMENT, INC."
1,AAPL,APPLE INC.
2,BAC,BANK OF AMERICA CORPORATION
3,AMZN,"AMAZON.COM, INC."
4,T,AT&T INC.
...,...,...
126758,SHCAU,SPINDLETOP HEALTH ACQUISITION CORP.
126759,ASPAW,"ABRI SPAC I, INC."
126760,ENERR,ACCRETION ACQUISITION CORP.
126761,ENERW,ACCRETION ACQUISITION CORP.


In [92]:
from whoosh.index import create_in

In [93]:
schema = Schema(Ticker = TEXT(stored=True), Name=TEXT(stored=True))

In [94]:
ix = create_in(".", schema)

In [95]:
writer = ix.writer()

In [96]:
# names = [Ticker, Name]

In [97]:
for index, row in df.iterrows():
    writer.add_document(Ticker = str(df.Ticker.iloc[index]),
                        Name   = str(df.Name.iloc[index])
                        )
writer.commit()

In [98]:
# for i in df.range():
#     writer.add_document(Ticker = str(df.title.iloc[i]), 
#                         Name = str(df.content.iloc[i]),
#                         )
# writer.commit()

In [99]:
from whoosh.qparser import QueryParser
from whoosh.query import *

In [100]:
Search_letter = 'a'

In [101]:
from whoosh import qparser

In [102]:
# search_fields = ['title','content']

# og = qparser.OrGroup.factory(0.9)
# mp = qparser.MultifieldParser(search_fields, schema, group = og)

# q = mp.parse(Search_letter)

# with ix.searcher() as s:
#     results = s.search(q, terms=True, limit = 10)
#     print("Search Results: ")
        
        
#     print(results[0:10])

In [103]:
# searcher = ix.searcher()
# proposed_time_strings = []
# with ix.searcher() as searcher:
#     parser = QueryParser("Name", ix.schema)
#     # querystring = Search_letter
#     querystring = u"CACI INTL INC"
#     myquery = parser.parse(querystring)
#     print(myquery)
#     results = searcher.search(myquery)
#     results.fragmenter.surround = 500
#     results.fragmenter.maxchars = 1500
#     print(len(results))
#     for res in results:
#         print(res['Ticker'])
#         #if(str.find(sub,start,end))
#         res_strings = res.highlights("Name", top = 5)
#         print(res_strings)
#         print("END__________________________________________________")

#         proposed_time_strings.append(res_strings)

In [104]:
def Search_Ticker(query):
   
    searcher = ix.searcher()
    proposed_time_strings = []
    with ix.searcher() as searcher:
        parser = QueryParser("Name", ix.schema)
        querystring = str(query)
        # querystring = u"Apple"
        myquery = parser.parse(querystring)
        print(myquery)
        results = searcher.search(myquery)
        results.fragmenter.surround = 500
        results.fragmenter.maxchars = 1500
        # print(len(results))
        if (len(results) >= 1):
            print("Ticker: " + results[0]['Ticker'])
            print("__________________________________________________")
            return results[0]['Ticker']
        else:
            print("Not match found")
            print("__________________________________________________")
            pass

        # for res in results:
        #     print(res['Ticker'])
            # #if(str.find(sub,start,end))
            # res_strings = res.highlights("Name", top = 1)
            # print(res_strings)
            # print("END__________________________________________________")

            # proposed_time_strings.append(res_strings)

        

In [105]:
df_kek['Ticker of Issuer 2'] = ""

In [106]:
df_kek.head(5)

,Unnamed: 0,CIK,Name of fund,Quater of report,Signing date,Name of Issuer,Ticker of Issuer,CUSIP,Value (x$1000),Shares,Shares type,Investment Discretion,Voting Sole,Shared,None,Ticker of Issuer 2
0,0,1584087,1492 Capital Management LLC,2022-03-31,2022-05-11,A10 NETWORKS INC,ATEN,002121101,188585,10041,SH,SOLE,10041,0,0,
1,1,1584087,1492 Capital Management LLC,2022-03-31,2022-05-11,ABBVIE INC,ABBV,00287Y109,140,1340,SH,SOLE,1340,0,0,
2,2,1584087,1492 Capital Management LLC,2022-03-31,2022-05-11,ACCEL ENTERTAINMENT INC,acel,00436Q106,217,287328,SH,SOLE,287328,0,0,
3,3,1584087,1492 Capital Management LLC,2022-03-31,2022-05-11,ADTRAN INC,ADTN,00738A106,3500,187092,SH,SOLE,187092,0,0,
4,4,1584087,1492 Capital Management LLC,2022-03-31,2022-05-11,AMERESCO INC,AMRC,02361E108,3452,10401,SH,SOLE,10401,0,0,


In [107]:
for index in range(len(df_kek)):
    df_kek['Ticker of Issuer 2'].iloc[index] = Search_Ticker(df_kek['Name of Issuer'].iloc[index])

(Name:a10 AND Name:networks AND Name:inc)
Ticker: ATEN
__________________________________________________
(Name:abbvie AND Name:inc)


C:\Users\vladm\AppData\Local\Temp\ipykernel_44988\3799809491.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_kek['Ticker of Issuer 2'].iloc[index] = Search_Ticker(df_kek['Name of Issuer'].iloc[index])


Ticker: ABBV
__________________________________________________
(Name:accel AND Name:entertainment AND Name:inc)
Ticker: ACEL
__________________________________________________
(Name:adtran AND Name:inc)
Ticker: ADTN
__________________________________________________
(Name:ameresco AND Name:inc)
Ticker: AMRC
__________________________________________________
(Name:american AND Name:vanguard AND Name:corporation)
Ticker: AVD
__________________________________________________
(Name:ameris AND Name:bancorp)
Ticker: ABCB
__________________________________________________
(Name:ammo AND Name:inc)
Ticker: RTRO
__________________________________________________
(Name:ani AND Name:pharmaceuticals AND Name:inc)
Ticker: ANIP
__________________________________________________
(Name:apollo AND Name:endosurgery AND Name:inc)
Ticker: APEN
__________________________________________________
(Name:arcutis AND Name:biotherapeutics AND Name:inc)
Ticker: ARQT
______________________________________________

In [108]:
df_kek.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108 entries, 0 to 107
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Unnamed: 0             108 non-null    int64 
 1   CIK                    108 non-null    int64 
 2   Name of fund           108 non-null    object
 3   Quater of report       108 non-null    object
 4   Signing date           108 non-null    object
 5   Name of Issuer         108 non-null    object
 6   Ticker of Issuer       108 non-null    object
 7   CUSIP                  108 non-null    object
 8   Value (x$1000)         108 non-null    int64 
 9   Shares                 108 non-null    int64 
 10  Shares type            108 non-null    object
 11  Investment Discretion  108 non-null    object
 12  Voting Sole            108 non-null    int64 
 13  Shared                 108 non-null    int64 
 14  None                   108 non-null    int64 
 15  Ticker of Issuer 2     

In [109]:
df_kek.head(100)

,Unnamed: 0,CIK,Name of fund,Quater of report,Signing date,Name of Issuer,Ticker of Issuer,CUSIP,Value (x$1000),Shares,Shares type,Investment Discretion,Voting Sole,Shared,None,Ticker of Issuer 2
0,0,1584087,1492 Capital Management LLC,2022-03-31,2022-05-11,A10 NETWORKS INC,ATEN,002121101,188585,10041,SH,SOLE,10041,0,0,ATEN
1,1,1584087,1492 Capital Management LLC,2022-03-31,2022-05-11,ABBVIE INC,ABBV,00287Y109,140,1340,SH,SOLE,1340,0,0,ABBV
2,2,1584087,1492 Capital Management LLC,2022-03-31,2022-05-11,ACCEL ENTERTAINMENT INC,acel,00436Q106,217,287328,SH,SOLE,287328,0,0,ACEL
3,3,1584087,1492 Capital Management LLC,2022-03-31,2022-05-11,ADTRAN INC,ADTN,00738A106,3500,187092,SH,SOLE,187092,0,0,ADTN
4,4,1584087,1492 Capital Management LLC,2022-03-31,2022-05-11,AMERESCO INC,AMRC,02361E108,3452,10401,SH,SOLE,10401,0,0,AMRC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,1584087,1492 Capital Management LLC,2022-03-31,2022-05-11,SUMMIT MATERIALS INC,No Symbol Found,86614U100,197,37610,SH,SOLE,37610,0,0,SUM
96,96,1584087,1492 Capital Management LLC,2022-03-31,2022-05-11,SYNEOS HEALTH INC,SYNH,87166B102,1168,49848,SH,SOLE,49848,0,0,SYNH
97,97,1584087,1492 Capital Management LLC,2022-03-31,2022-05-11,TANDEM DIABETES CARE INC,TNDM,875372203,4035,10372,SH,SOLE,10372,0,0,TNDM
98,98,1584087,1492 Capital Management LLC,2022-03-31,2022-05-11,THIRD COAST BANCSHARES INC,TCBX,88422P109,1206,52214,SH,SOLE,52214,0,0,TCBX


In [111]:
df_kek.to_csv(r'D:\GitHub\Yahoo-project\export.csv', index = False, header=True)